<a href="https://colab.research.google.com/github/lhuang-pvamu/Parallel-Computing-Code/blob/master/Python/Numba_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numba Tutorial for GPU-Cuda

## Universal function: ufuncs

In [1]:
import numpy as np

In [2]:
a = np.array([1,2,3,4])
b = np.array([10,20,30,40])

In [3]:
np.add(a,b) # ufunc

array([11, 22, 33, 44])

In [4]:
np.add(a,1)

array([2, 3, 4, 5])

In [5]:
c = np.arange(16).reshape((4,4))
c

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [6]:
np.add(b,c)

array([[10, 21, 32, 43],
       [14, 25, 36, 47],
       [18, 29, 40, 51],
       [22, 33, 44, 55]])

In [46]:
from numba import jit, vectorize, cuda

In [44]:
@vectorize(['int64(int64, int64)','float32(float32, float32)'], target='cuda')
def add_ufunc(x,y):
  return x+y

In [13]:
add_ufunc(a,b)

array([11, 22, 33, 44])

In [14]:
a_list=[1,2,3,4]
b_list=[10,20,30,40]

In [20]:
%timeit add_ufunc(c, b)  # run on GPU

1000 loops, best of 5: 1.2 ms per loop


In [21]:
%timeit np.add(c,b)  # run on CPU

The slowest run took 33.73 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 5: 1.13 µs per loop


In [42]:
n = 100000
x = np.arange(n).astype(np.float32)
y = 2 * x

In [45]:
%timeit add_ufunc(x,y)

The slowest run took 91.57 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 1.68 ms per loop


## Device memory management

In [49]:
x_device = cuda.to_device(x)  # copy x from cpu to gpu memory
y_device = cuda.to_device(y)  # copy y from cpu to gpu memory
print(x_device)

In [50]:
%timeit add_ufunc(x_device,y_device)

1000 loops, best of 5: 729 µs per loop


In [51]:
out_device = cuda.device_array(shape=(n,), dtype=np.float32)

In [53]:
%timeit add_ufunc(x_device, y_device, out=out_device)

The slowest run took 4.36 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 613 µs per loop


In [55]:
out_host = out_device.copy_to_host()
print(out_host)

[0.00000e+00 3.00000e+00 6.00000e+00 ... 2.99991e+05 2.99994e+05
 2.99997e+05]


## One statistics example

In [39]:
import math 
SQRT_2PI = np.float32((2*math.pi)**0.5)

# Gaussian probability density function
@vectorize(['float32(float32, float32, float32)', 'int64(int64,int64,int64)'], target='cuda')
def gaussian_pdf(x, mean, sigma):
  return math.exp(-0.5 * ((x - mean) / sigma)**2) / (sigma * SQRT_2PI)

In [40]:
# Evaluate the Gaussian a million times!
x = np.random.uniform(-3, 3, size=1000000).astype(np.float32)
mean = np.float32(0.0)
sigma = np.float32(1.0)

In [41]:
%timeit gaussian_pdf(x, mean, sigma)

The slowest run took 36.47 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 5: 5.03 ms per loop


In [28]:
import scipy.stats # for definition of gaussian distribution
norm_pdf = scipy.stats.norm
%timeit norm_pdf.pdf(x, loc=mean, scale=sigma)

10 loops, best of 5: 54.7 ms per loop
